<a href="https://colab.research.google.com/github/hhdjwdabsxsx/Medical-Diagnosis-using-LSTM/blob/main/Medical_Diagnosis_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Medical Diagnosis with LSTM**
Building a deep learning model for medical diagnosis requires a large dataset of labeled medical data. The
dataset used in this tutorial includes:



*   Patient Symptoms: Textual descriptions of the patients symptoms.
*   Diagnoses: The confirmed diseases for each patient.

*   Medications: The prescribed medications for each patient's condition.

The deep learning model is built using TensorFlow, a popular open-source library for machine learning. The model architecture utilizes an LSTM layer to process the sequence of tokens representing the patient's symptoms. The output of the LSTM layer is then fed into two separate dense layers, one for predicting the disease and another for predicting the medication.





**Importing Libraries**

First we will import all the necessary libraries for handling data. 'Tokenizer' from  tensroflow will be used for text tokenization, 'pad_sequences' will be used for sequence paddi ng. 'to_categorical' will be used for converting labels to binary class matrices, and the 'LabelEncoder' from scikit-learn will be used for encoding text labels as integers.

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

**Loading the Dataset**

In [ ]:
data = pd.read_csv('medical_data.csv')
data.head()


,Patient_Problem,Disease,Prescription
0,"Constant fatigue and muscle weakness, struggli...",Chronic Fatigue Syndrome,"Cognitive behavioral therapy, graded exercise ..."
1,"Frequent severe migraines, sensitivity to ligh...",Migraine with Aura,"Prescription triptans, avoid triggers like bri..."
2,"Sudden weight gain and feeling cold, especiall...",Hypothyroidism,Levothyroxine to regulate thyroid hormone levels.
3,"High fever, sore throat, and swollen lymph nod...",Mononucleosis,"Rest and hydration, ibuprofen for pain."
4,"Excessive thirst and frequent urination, dry m...",Diabetes Mellitus,Insulin therapy and lifestyle changes.


**Data Preprocessing and Preparation**

Before using medical data in a deep learning model, it needs to be preprocessed to ensure the model can understand it. Preprocessing steps often include:

*   Text Tokenization: Converting textual data into sequences of numbers that the model can process.
*   Padding sequences: Making all sequemces the same length by adding padding characters at the beginning or end of shorter sequences.

*   Label Encoding: COnverting categorical variables such as disease names and medication names, into numerical labels.




**Tokenization and Sequencing Text Data**

In [ ]:
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(data['Patient_Problem'])

sequences = tokenizer.texts_to_sequences(data['Patient_Problem'])

A 'tokenizer' variable is created to convert the textual data into sequences of integers. It only considers the top 5,000 words in the dataset in order to reduce the complexity. If the model encounters an out-of-vocabulary words during the tyraining process then it will be replaced with "<OOV>" token.

**Padding Sequences**

In order to make the input sequences have the same length, the code finds the lengest sequence and pads all other sequences with zeros at the end ('post' padding) to match this sentence.

In [ ]:
max_length = max(len(x) for x in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

**Encoding the Labels and Converting them to Categorical**
we will encode the 'Disease' and 'Prescription' column as integers. Then the integer-encoded labels are converted into binary class matrices.

In [ ]:
# Encoding the labels
label_encoder_disease = LabelEncoder()
label_encoder_prescription = LabelEncoder()

disease_labels = label_encoder_disease.fit_transform(data['Disease'])
prescription_labels = label_encoder_prescription.fit_transform(data['Prescription'])

# Converting labels to categorical
disease_labels_categorical = to_categorical(disease_labels)
prescription_labels_categorical = to_categorical(prescription_labels)

**Combining Labels into a Multi-label Target Variable**

Finally. now we will stack the binary class matrices together to form a single multi-label target variable 'Y'. This allows the model to predict both 'Disease' and 'Prescription' from the patient's problem.

In [ ]:
Y = np.hstack((disease_labels_categorical, prescription_labels_categorical))

**Model Building**

Now, we will build the model using the LSTM and Sequential algorithm from TensorFLow. This model will learn from our preprocessed dataset to predict diseases based on patient symptoms.

**Defining Model Architecture**

We will use the 'Model' and 'Input' to define the model architecture, and 'Embedding' to convert the integer sequences into dense vectors of fixed size. We will use 'Dense' for output layers that make predictions.

In [ ]:
input_layer = Input(shape=(max_length,))

embedding = Embedding(input_dim=5000, output_dim=64)(input_layer)
lstm_layer = LSTM(64)(embedding)

disease_output = Dense(len(label_encoder_disease.classes_), activation='softmax', name='disease_output')(lstm_layer)

prescription_output = Dense(len(label_encoder_prescription.classes_), activation='softmax', name='prescription_output')(lstm_layer)

The model firstly has, an input layer that can handle sequnce up to a certain length. Then there's an embedding layer that turns the numbers into vectors. After that, there's an LSTM layer that looks at the order of things and finally, two dense layers that predict diseases and prescriptions using a softmax function or classification.

**Compiling the Model**

In [ ]:
model = Model(inputs=input_layer, outputs=[disease_output, prescription_output])

model.compile(
    loss={'disease_output': 'categorical_crossentropy',
          'prescription_output': 'categorical_crossentropy'},
          optimizer='adam',
          metrics={'disease_output': ['accuracy'], 'prescription_output': ['accuracy']}
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 17)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 17, 64)         │        320,000 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 64)             │         33,024 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ disease_output (Dense)    │ (None, 178)            │         11,570 │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ prescription_output       │ (None, 388)            │         25,220 │ lstm[0][0]             │
│ (Dense)                   │                        │                │                        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 389,814 (1.49 MB)

 Trainable params: 389,814 (1.49 MB)

 Non-trainable params: 0 (0.00 B)

**Training the Model**


In [ ]:
model.fit(padded_sequences, {'disease_output': disease_labels_categorical, 'prescription_output':
      prescription_labels_categorical}, epochs=100, batch_size=32)

Epoch 1/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - disease_output_accuracy: 8.8801e-04 - disease_output_loss: 5.1826 - loss: 11.1456 - prescription_output_accuracy: 0.0034 - prescription_output_loss: 5.9631
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - disease_output_accuracy: 0.0442 - disease_output_loss: 5.1672 - loss: 11.1277 - prescription_output_accuracy: 0.0000e+00 - prescription_output_loss: 5.9605
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - disease_output_accuracy: 0.0202 - disease_output_loss: 5.1428 - loss: 11.1033 - prescription_output_accuracy: 0.0018 - prescription_output_loss: 5.9606
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - disease_output_accuracy: 0.0187 - disease_output_loss: 5.0310 - loss: 10.9937 - prescription_output_accuracy: 0.0124 - prescription_output_loss: 5.9634
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - disease_output_accuracy: 0.0231 - disease_output_loss: 4.8961 - loss: 10.8512 - prescription_output_accuracy: 0.00

**Making Predictions**

The model is used to make predictions for many patients:

1. Pre-processed the patient's symptoms by performing tokenization and padding.

2. Feed the pre-processed data into the trained model.

3. The model predicts the disease and medication based on the patient's symptoms.

4. The predicted disease and medication will be presented.

In [ ]:
def make_prediction(patient_problem):
    # Preprocessing the input
    sequence = tokenizer.texts_to_sequences([patient_problem])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post')

    # Making the prediction
    # prediction variable was missing, added it here using the model.predict function.
    prediction = model.predict(padded_sequence)
    disease_index = np.argmax(prediction[0], axis=1)[0]
    prescription_index = np.argmax(prediction[1], axis=1)[0]

    disease_predicted = label_encoder_disease.inverse_transform([disease_index])[0]
    prescription_predicted = label_encoder_prescription.inverse_transform([prescription_index])[0]

    print(f"Predicted Disease: {disease_predicted}")
    print(f"Predicted Prescription: {prescription_predicted}")

patient_input = "I've experienced a loss of appetite and don't enjoy food anymore."
make_prediction(patient_input)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Predicted Disease: Depression
Predicted Prescription: Antidepressants; eating nutrient-rich foods.


**Save the model**

In [ ]:
# save model
model.save('model.keras')